In [5]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib as mt
from dateutil.relativedelta import *

np.set_printoptions(suppress=True)

In [6]:
def compute_daily_returns(df):
    daily_returns=(df/df.shift(1))-1
    df=df.fillna(value=0)
    daily_returns=daily_returns[1:]
    return daily_returns



In [7]:
def var_calculator(data_frame, start_date, end_date):
    value_at_risk_matrix = []
    returns_daily = compute_daily_returns(data_frame)
    
    for symbol in kse30_symbols:
        returns_matrix = returns_daily.loc[start_date : end_date,'{}'.format(symbol)]
        return_matrix = np.array(returns_matrix)
        value_at_risk = np.percentile(return_matrix, 100 * (1-0.99))
        #print(value_at_risk)
        cvarcalc = np.nanmean(returns_matrix < value_at_risk)
        #print(cvarcalc)
        value_at_risk_matrix.append(cvarcalc)
    var_df = pd.DataFrame(data = value_at_risk_matrix, index=kse30_symbols)
    return var_df



In [8]:
def stock_picker(data_frame, start_date, start_date_test, end_date_test,number_of_stocks_selected,principal_amount):
    var_df = var_calculator(data_frame, start_date, start_date_test)
    var_df.sort_values(by = 0 ,axis = 0, ascending = False, inplace = True)
    
    ######### Calculating for the best combination ########################
    amount_in_one = principal_amount / number_of_stocks_selected
    
    symbols_to_invest = var_df.index.values.tolist()[0:number_of_stocks_selected]
    symbols_to_invest_df = data_frame.loc[start_date_test:end_date_test ,symbols_to_invest]
    symbols_to_invest_returns = compute_daily_returns(symbols_to_invest_df)
    symbols_to_invest_returns = symbols_to_invest_returns + 1
    symbols_to_invest_returns.iloc[0,:] = symbols_to_invest_returns.iloc[0,:] * amount_in_one    
    symbols_to_invest_returns = symbols_to_invest_returns.cumprod()
    symbols_to_invest_returns['Aggregated_Return_Combination'] = symbols_to_invest_returns.iloc[:, 0:].sum(axis=1)
    symbols_to_invest_returns = symbols_to_invest_returns.iloc[:,number_of_stocks_selected:]
    
    ########## Calculating for all stocks ##########################
    amount_in_all = principal_amount / len(data_frame.columns)
    
    symbols_all = var_df.index.values.tolist()[0:len(data_frame.columns)]
    symbols_all_df = data_frame.loc[start_date_test:end_date_test ,symbols_all]
    symbols_all_returns = compute_daily_returns(symbols_all_df)
    symbols_all_returns = symbols_all_returns + 1
    symbols_all_returns.iloc[0,:] = symbols_all_returns.iloc[0,:] * amount_in_all
    symbols_all_returns = symbols_all_returns.cumprod()
    symbols_all_returns['Aggregated_Return_all']= symbols_all_returns.iloc[:, 0:].sum(axis=1)
    symbols_all_returns = symbols_all_returns.iloc[:,len(data_frame.columns):]
    
    ########## Merging Both and returning ##########################
    df = symbols_all_returns.merge(symbols_to_invest_returns,how='outer', left_index=True, right_index=True)
    
    return df

In [9]:
df = pd.read_csv('E:\\Stock\\Data\\MCBAH\\NAVS.csv')
df = df.iloc[:,1:]
df.iloc[:,0] = pd.to_datetime(df.iloc[:,0])
df = df.set_index('Date')

In [10]:
kse30_symbols = list(df)

In [23]:
funds_returns = stock_picker(df,'2018-01-01','2018-11-01','2019-11-30',3,100000)
principal_amount = 100000
#funds_returns.sum()


C:\ProgramData\Anaconda3\lib\site-packages\numpy\lib\function_base.py:3652: RuntimeWarning: Invalid value encountered in percentile
  interpolation=interpolation)


In [24]:
funds_returns

,Aggregated_Return_all,Aggregated_Return_Combination
Date,,
2018-11-02,100152.930697,100019.459817
2018-11-05,99891.583528,100042.055775
2018-11-06,99579.306185,100047.963587
2018-11-07,99983.589829,100048.281976
2018-11-08,99829.515510,99957.670472
2018-11-09,99865.501936,99977.916503
2018-11-12,99719.710966,100045.332646
2018-11-13,99781.793114,100049.688894
2018-11-14,99586.522182,100059.843316


In [18]:
def back_tester(data_frame,start_date,no_of_training_months,number_of_stocks_selected,principal_amount):
    start_date = pd.to_datetime(start_date)
    start_date_test =  start_date+relativedelta(months=+no_of_training_months)
    #start_date_test =  start_date+relativedelta(days=+1)
    end_date_test = start_date_test+relativedelta(months=+1)
    funds_returns = stock_picker(df,start_date,start_date_test,end_date_test,number_of_stocks_selected,principal_amount)
    updated_amount = funds_returns.iloc[funds_returns.shape[0]-1:,1:]
    return updated_amount,funds_returns

In [20]:
updated_amount, funds_returns = back_tester(df,'2018-01-01',10,3,100000)

C:\ProgramData\Anaconda3\lib\site-packages\numpy\lib\function_base.py:3652: RuntimeWarning: Invalid value encountered in percentile
  interpolation=interpolation)


In [21]:
updated_amount

,Aggregated_Return_Combination
Date,
2018-11-30,100246.653619


In [22]:
funds_returns

,Aggregated_Return_all,Aggregated_Return_Combination
Date,,
2018-11-02,100152.930697,100019.459817
2018-11-05,99891.583528,100042.055775
2018-11-06,99579.306185,100047.963587
2018-11-07,99983.589829,100048.281976
2018-11-08,99829.515510,99957.670472
2018-11-09,99865.501936,99977.916503
2018-11-12,99719.710966,100045.332646
2018-11-13,99781.793114,100049.688894
2018-11-14,99586.522182,100059.843316


In [57]:
max_date = df.index.max()
min_date = df.index.min()
no_of_months_in_data = (max_date.year - min_date.year) * 12 + max_date.month - min_date.month
updated_amount = 100000


In [ ]:
Start_month = 6
for x in range(Start_month, no_of_months_in_data+1):
    updated_amount, funds_returns = back_tester(df,min_date,x,3,updated_amount)